In [12]:
import geopandas as gpd
import pandas as pd
import json
from shapely.geometry import mapping, shape
from shapely.prepared import prep
from shapely.geometry import Point

data = gpd.read_file(r'C:\Users\ogechi\OneDrive - University of Glasgow\bottle\countries.geojson')
df = pd.read_json(r'C:\Users\ogechi\OneDrive - University of Glasgow\bottle\processed_image_data.indoor.json')

['ADMIN', 'ISO_A3', 'ISO_A2', 'geometry']

In [16]:
data['country_bounds'] = data.apply(lambda row: prep(shape(row['geometry'])), axis=1)
data.head()

In [17]:
data.head()

,ADMIN,ISO_A3,ISO_A2,geometry,country_bounds
0,Aruba,ABW,AW,"MULTIPOLYGON (((-69.99694 12.57758, -69.93639 ...",<shapely.prepared.PreparedGeometry object at 0...
1,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38...",<shapely.prepared.PreparedGeometry object at 0...
2,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...",<shapely.prepared.PreparedGeometry object at 0...
3,Anguilla,AIA,AI,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ...",<shapely.prepared.PreparedGeometry object at 0...
4,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42...",<shapely.prepared.PreparedGeometry object at 0...


In [34]:
def get_country(lon, lat):
    if pd.isna(lon) or pd.isna(lat):
        return ""
    point = Point(lon, lat)
    for i, row in data.iterrows():
        geom = row['country_bounds']
        if geom.contains(point):
            return row['ADMIN']
    return ""


df['country_label'] = df.apply(lambda row: get_country(row['longitude'], row['latitude']), axis=1)

In [35]:
df.head()

,_id,title,description,license,date_upload,date_taken,owner_name,original_format,last_update,latitude,...,photoid,accuracy,page_url,licenseurl,isindoor,category,page_source,address,type,country_label
0,{'$oid': '66031c7b2008c048c986ec6b'},20121227_IMG_2406,,4,1.359064e+09,2012-12-27 11:35:01,54177777@N00,jpg,1359064484,35.662359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Japan
1,{'$oid': '66031c7b2008c048c986ec6c'},20121227_IMG_2405,,4,1.359064e+09,2012-12-27 11:34:08,54177777@N00,jpg,1359064484,35.662359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Japan
2,{'$oid': '66031c7b2008c048c986ec6d'},20121227_IMG_2404,,4,1.359064e+09,2012-12-27 11:34:02,54177777@N00,jpg,1359064483,35.662359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Japan
3,{'$oid': '66031c7b2008c048c986ec6e'},20121227_IMG_2399,,4,1.359064e+09,2012-12-27 11:31:54,54177777@N00,jpg,1359064483,35.662359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Japan
4,{'$oid': '66031c7b2008c048c986ec6f'},20121227_IMG_2398,,4,1.359064e+09,2012-12-27 11:31:34,54177777@N00,jpg,1359064483,35.662359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Japan


In [36]:
# Save the updated DataFrame to a new CSV file
df.to_csv('updated_dataframe.csv', index=False)

In [ ]:
import pandas as pd
import plotly.express as px

# Read the data into a pandas DataFrame
df = pd.read_csv('your_data.csv')

# Optimize the code by grouping the data by country_name and counting the occurrences
country_counts = df.groupby('country_name').size().reset_index(name='counts')

# Create the choropleth map
fig = px.choropleth(country_counts,
                    locations='country_name',
                    color='counts',
                    hover_name='country_name',
                    color_continuous_scale='Plasma',
                    projection='natural earth')

# Update the layout
fig.update_layout(
    title='Number of Data Points per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

# Show the map
fig.show()

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Load your dataframes
df = pd.read_json(r'C:\Users\ogechi\OneDrive - University of Glasgow\bottle\processed_image_data.indoor.json')
data = gpd.read_file(r'C:\Users\ogechi\OneDrive - University of Glasgow\bottle\countries.geojson')  # Ensure this file contains the necessary columns and geometries.
data['country_bounds'] = data.apply(lambda row: prep(shape(row['geometry'])), axis=1)
# Ensure 'data' has a proper GeoDataFrame setup with prepared geometries
data['geom'] = data['country_bounds'].apply(lambda g: g.geom)  # Convert PreparedGeometry to usual Geometry if needed.
countries_gdf = gpd.GeoDataFrame(data, geometry='geom')

# Convert df to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")

# Spatial join between points and countries
result = gpd.sjoin(gdf, countries_gdf, how="left", op='within')

# Drop the unnecessary geometry column if no longer needed
result = result.drop(columns=['geometry', 'index_right'])

# Map the 'ADMIN' column from countries to your dataframe
df['country_label'] = result['ADMIN']

# Save or use df as needed


In [37]:
import numpy as np
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Load your dataset
data = pd.read_json(r'C:\Users\ogechi\OneDrive - University of Glasgow\bottle\processed_image_data.indoor.json')

In [2]:
summary_width = data['width'].describe()
summary_height = data['height'].describe()

print("Width Summary:")
print(summary_width)
print("\nHeight Summary:")
print(summary_height)

Width Summary:
count    320720.000000
mean       3270.408525
std        1602.419681
min          71.000000
25%        2048.000000
50%        3264.000000
75%        4032.000000
max       34755.000000
Name: width, dtype: float64

Height Summary:
count    320720.000000
mean       2608.254721
std        1244.514887
min          54.000000
25%        1836.000000
50%        2591.000000
75%        3240.000000
max       19600.000000
Name: height, dtype: float64


In [3]:
# Combine width and height into a tuple and count occurrences
common_dimensions = data.groupby(['width', 'height']).size().reset_index(name='count')
common_dimensions = common_dimensions.sort_values(by='count', ascending=False)

print("Most Common Dimensions:")
print(common_dimensions.head(10))  # Display top 10 most common dimensions


Most Common Dimensions:
        width  height  count
27795  3264.0  2448.0  32142
37043  4032.0  3024.0  16060
36412  4000.0  3000.0  10520
42787  5312.0  2988.0   8957
13782  2048.0  1365.0   6750
42482  5184.0  3456.0   5938
44711  6000.0  4000.0   5901
32423  3648.0  2736.0   5387
2068    640.0   640.0   4393
24810  3024.0  4032.0   4384


In [4]:
# Calculate aspect ratios
data['aspect_ratio'] = data['width'] / data['height']

# Summary of aspect ratios
aspect_ratio_summary = data['aspect_ratio'].describe()

print("Aspect Ratio Summary:")
print(aspect_ratio_summary)

# Frequency of common aspect ratios
common_aspect_ratios = data['aspect_ratio'].value_counts().head(10)
print("\nCommon Aspect Ratios:")
print(common_aspect_ratios)


Aspect Ratio Summary:
count    320720.000000
mean          1.288868
std           0.366547
min           0.125151
25%           1.000000
50%           1.333333
75%           1.500000
max          15.000000
Name: aspect_ratio, dtype: float64

Common Aspect Ratios:
1.333333    99612
1.500000    30503
1.777778    24297
0.750000    20655
1.000000    20284
0.666667    10093
1.500366     6759
1.499268     6141
1.338843     3171
1.493827     3047
Name: aspect_ratio, dtype: int64


In [38]:
def convert_dates(date_item):
    # Check the type of the date_item
    if isinstance(date_item, pd.Timestamp):
        # Already a Timestamp, return as is
        return date_item
    try:
        # Assume it's a string and check if it's all digits (Unix timestamp)
        if isinstance(date_item, str) and date_item.isdigit():
            # Convert Unix timestamp to datetime
            return pd.to_datetime(date_item, unit='s')
        elif isinstance(date_item, str):
            # Convert standard datetime strings
            return pd.to_datetime(date_item)
    except Exception as e:
        # Handle exceptions that might be raised during conversion
        print(f"Error converting {date_item}: {e}")
        return pd.NaT  # Return NaT (Not a Time) for unconvertible formats

# Apply the conversion function to each date column
data['date_upload'] = data['date_upload'].apply(convert_dates)
data['date_taken'] = data['date_taken'].apply(convert_dates)
data['last_update'] = data['last_update'].apply(convert_dates)

# Statistical Summary
summary_upload = data['date_upload'].agg(['min', 'max'])
summary_taken = data['date_taken'].agg(['min', 'max'])
summary_update = data['last_update'].agg(['min', 'max'])

Error converting 0001-01-01 12:12:12.0: Out of bounds nanosecond timestamp: 1-01-01 12:12:12
Error converting null: Unknown string format: null


C:\Users\ogechi\anaconda3\envs\pytorch\lib\site-packages\pandas\core\arrays\datetimelike.py:1189: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  warnings.warn(


TypeError: unsupported operand type(s) for -: 'Timestamp' and 'NoneType'

In [39]:
# Calculate time intervals
data['upload_to_taken'] = data.apply(
    lambda row: row['date_taken'] - row['date_upload'] if pd.notna(row['date_taken']) and pd.notna(row['date_upload']) else pd.NaT,
    axis=1
)

data['last_update_to_now'] = data.apply(
    lambda row: pd.Timestamp('now') - row['last_update'] if pd.notna(row['last_update']) else pd.NaT,
    axis=1
)

# Output summary statistics
print("Upload Dates Summary:", summary_upload)
print("Taken Dates Summary:", summary_taken)
print("Update Dates Summary:", summary_update)
print("\nCalculated Time Intervals:")
print(data[['upload_to_taken', 'last_update_to_now']])

Upload Dates Summary: min   NaN
max   NaN
Name: date_upload, dtype: float64
Taken Dates Summary: min   1825-01-01 00:00:00
max   2099-12-27 18:00:11
Name: date_taken, dtype: datetime64[ns]
Update Dates Summary: min   1970-01-01 00:00:00
max   2024-04-23 14:06:00
Name: last_update, dtype: datetime64[ns]

Calculated Time Intervals:
       upload_to_taken        last_update_to_now
0                  NaT 4116 days 11:36:28.293254
1                  NaT 4116 days 11:36:28.293254
2                  NaT 4116 days 11:36:29.293254
3                  NaT 4116 days 11:36:29.293254
4                  NaT 4116 days 11:36:29.293254
...                ...                       ...
677725             NaT                       NaT
677726             NaT                       NaT
677727             NaT                       NaT
677728             NaT                       NaT
677729             NaT                       NaT

[677730 rows x 2 columns]


In [43]:
# Determine non-empty and non-null descriptions
non_empty_descriptions = data['title'].apply(lambda x: isinstance(x, str) and x.strip() != '')

# Calculate the percentage of non-empty/non-null descriptions
percentage_with_descriptions = (non_empty_descriptions.sum() / len(data) * 100)

print(f"Percentage of data with descriptions: {percentage_with_descriptions:.2f}%")

Percentage of data with descriptions: 91.91%


In [33]:
from wordcloud import WordCloud
from collections import Counter

wc = WordCloud()

counts_all = Counter()

with open('example.txt', 'r') as f:
    for line in f:  # Here you can also use the Cursor
        counts_line = wc.process_text(line)
        counts_all.update(counts_line)

wc.generate_from_frequencies(counts_all)
wc.to_file('wc.png')

ValueError: Only supported for TrueType fonts